<a href="https://colab.research.google.com/github/SiddhiMarri/PostureDetection/blob/main/Detect.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import re

def parse_results_file(file_path):
    frame_data = {}

    with open(file_path, "r") as file:
        lines = file.readlines()

        for line in lines:
            # Use regex to extract the frame number and keypoints
            match = re.match(r"(\d+), \[\[\[(.*?)\]\]\]", line.strip())
            if match:
                frame = int(match.group(1))  # Extract frame number
                keypoints_raw = match.group(2)  # Extract raw keypoints string
                keypoints = eval("[" + keypoints_raw + "]")  # Convert to list of lists
                frame_data[frame] = keypoints

    return frame_data

# Example usage
results_file = "detection_results.txt"  # Path to the text file containing YOLO pose results
keypoints_by_frame = parse_results_file(results_file)

# Output a few frames to verify
for frame, keypoints in list(keypoints_by_frame.items()):
    print(f"Frame {frame}: {keypoints}")


Frame 0: ([1080.3060302734375, 281.05908203125], [1095.800048828125, 269.43621826171875], [1070.4627685546875, 268.0379638671875], [1124.3280029296875, 278.2651672363281], [1057.394775390625, 274.9235534667969], [1155.278076171875, 344.61993408203125], [1030.412353515625, 335.22247314453125], [1164.8046875, 421.918212890625], [995.44580078125, 397.9489440917969], [0.0, 0.0], [1051.476806640625, 346.73614501953125], [0.0, 0.0], [0.0, 0.0], [0.0, 0.0], [0.0, 0.0], [0.0, 0.0], [0.0, 0.0])
Frame 1: ([947.2517700195312, 290.90081787109375], [949.2177124023438, 278.8626708984375], [933.0780029296875, 278.95281982421875], [0.0, 0.0], [894.09814453125, 286.0540771484375], [951.998046875, 349.5616455078125], [874.3111572265625, 353.7664489746094], [0.0, 0.0], [0.0, 0.0], [0.0, 0.0], [0.0, 0.0], [0.0, 0.0], [0.0, 0.0], [0.0, 0.0], [0.0, 0.0], [0.0, 0.0], [0.0, 0.0])
Frame 2: ([1078.1063232421875, 277.97705078125], [1091.4658203125, 266.08404541015625], [1069.0167236328125, 266.22723388671875], [

In [6]:
import os
import math
import cv2

def classify_pose(keypoints):
    """
    Classifies the pose as 'Good Pose' or 'Bad Pose' based on keypoints.
    """
    left_shoulder = keypoints[5]
    right_shoulder = keypoints[6]
    left_hip = keypoints[11]
    right_hip = keypoints[12]

    x5, y5 = left_shoulder
    x6, y6 = right_shoulder
    x11, y11 = left_hip
    x12, y12 = right_hip

    # 1. Shoulder and hip alignment
    shoulder_alignment = abs(y5 - y6)
    hip_alignment = abs(y11 - y12)

    # 2. Vertical alignment
    shoulder_mid_x = (x5 + x6) / 2
    hip_mid_x = (x11 + x12) / 2
    vertical_alignment = abs(shoulder_mid_x - hip_mid_x)

    # 3. Calculate spinal angle
    def calculate_angle(vector1, vector2):
        dot_product = vector1[0] * vector2[0] + vector1[1] * vector2[1]
        magnitude1 = math.sqrt(vector1[0]**2 + vector1[1]**2)
        magnitude2 = math.sqrt(vector2[0]**2 + vector2[1]**2)
        if magnitude1 == 0 or magnitude2 == 0:
            return 0
        return math.acos(dot_product / (magnitude1 * magnitude2)) * (180 / math.pi)

    shoulder_vector = (x6 - x5, y6 - y5)
    hip_vector = (x12 - x11, y12 - y11)
    spinal_angle = calculate_angle(shoulder_vector, hip_vector)

    # Pose thresholds
    max_alignment_threshold = 15
    min_spinal_angle = 160
    max_spinal_angle = 200

    # Classification logic
    if (
        shoulder_alignment < max_alignment_threshold
        and hip_alignment < max_alignment_threshold
        and vertical_alignment < max_alignment_threshold
        and min_spinal_angle <= spinal_angle <= max_spinal_angle
    ):
        return "Good Pose"
    else:
        return "Bad Pose"

def draw_keypoints(frame, keypoints, pose_classification):
    """
    Draws keypoints and connections on the frame along with pose classification.
    """
    point_color = (0, 0, 255)  # Red for keypoints
    line_color = (255, 0, 0)  # Blue for connections
    thickness = 2
    radius = 5

    # Draw keypoints
    for (x, y) in keypoints:
        if x == 0.0 and y == 0.0:  # Skip missing keypoints
            continue
        cv2.circle(frame, (int(x), int(y)), radius, point_color, -1)

    # Draw connections
    connections = [
        (5, 6), (5, 11), (6, 12), (11, 12),  # Torso connections
        (5, 7), (7, 9), (6, 8), (8, 10),    # Arms
        (11, 13), (13, 15), (12, 14), (14, 16),  # Legs
    ]
    for i, j in connections:
        if keypoints[i][0] != 0.0 and keypoints[j][0] != 0.0:  # Only draw valid connections
            cv2.line(
                frame,
                (int(keypoints[i][0]), int(keypoints[i][1])),
                (int(keypoints[j][0]), int(keypoints[j][1])),
                line_color,
                thickness,
            )

    # Overlay pose classification text
    cv2.putText(
        frame,
        pose_classification,
        (50, 50),
        cv2.FONT_HERSHEY_SIMPLEX,
        1,
        (0, 255, 0) if pose_classification == "Good Pose" else (0, 0, 255),
        2,
        cv2.LINE_AA,
    )

def parse_results_file(results):
    frame_data = {}

    for frame, keypoints in results.items():
        # Ensure keypoints have the correct format
        valid_keypoints = [
            (float(x), float(y)) if x != 0.0 and y != 0.0 else (0.0, 0.0)
            for x, y in keypoints
        ]
        frame_data[frame] = valid_keypoints

    return frame_data

def process_video_with_keypoints(video_path, keypoints_data, output_path):
    """
    Processes a video, overlays keypoints and pose classification, and saves the output video.
    """
    cap = cv2.VideoCapture(video_path)

    # Get video properties
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = int(cap.get(cv2.CAP_PROP_FPS))
    fourcc = cv2.VideoWriter_fourcc(*"mp4v")  # Codec for output video

    # Initialize VideoWriter
    out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))

    frame_index = 0
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        # Get keypoints for the current frame
        keypoints = keypoints_data.get(frame_index, None)
        if keypoints:
            # Classify pose
            pose_classification = classify_pose(keypoints)

            # Draw keypoints and pose classification
            draw_keypoints(frame, keypoints, pose_classification)

        # Write the frame to the output video
        out.write(frame)

        frame_index += 1

    cap.release()
    out.release()
    print(f"Processed video saved to {output_path}")

# Example usage
results_file = keypoints_by_frame  # Path to the text file
video_path = "output_video (1).mp4"  # Path to the input video
output_video_path = "output_video_withgoodbad.mp4"  # Path to save the output video

# Parse keypoints from results file
keypoints_data = parse_results_file(results_file)

# Process video and overlay keypoints
process_video_with_keypoints(video_path, keypoints_data, output_video_path)


Processed video saved to output_video_withgoodbad.mp4
